In [264]:
import sqlite3
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import numpy as np
from geopy.distance import distance
import re
# from  keys  import  client_id, api_key
import folium

In [265]:
pd.set_option('display.max_rows', 200)

In [266]:
from cleaning_utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [267]:
blight = pd.read_csv('../data/Blight_Violations.csv')

/Users/kelvinarellano/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,11,15,16,17,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [268]:
blight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496335 entries, 0 to 496334
Data columns (total 36 columns):
X                             306227 non-null float64
Y                             306227 non-null float64
ticket_id                     496335 non-null int64
ticket_number                 496335 non-null object
agency_name                   496335 non-null object
inspector_name                496335 non-null object
violator_name                 496333 non-null object
violation_street_number       496335 non-null int64
violation_street_name         496270 non-null object
violation_zip_code            154162 non-null object
violator_id                   496335 non-null int64
mailing_address_str_number    496329 non-null object
mailing_address_str_name      496325 non-null object
city                          493079 non-null object
state                         492616 non-null object
zip_code                      493076 non-null object
non_us_str_code               3259 non-nul

In [269]:
MCM = pd.read_csv('../data/Motor_City_Mapping,_Winter_2013-14_Certified_Results.csv')

/Users/kelvinarellano/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [270]:
MCM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379549 entries, 0 to 379548
Data columns (total 33 columns):
OBJECTID             379549 non-null int64
D3_SurveyParcelID    379549 non-null object
CityParcelID2013     379549 non-null object
AddCombo             379549 non-null object
Address_Number       379549 non-null object
Address_Direction    379549 non-null object
Street               379549 non-null object
Zip                  379549 non-null object
PCount               379549 non-null int64
Blext_ID             377209 non-null float64
User_ID              373229 non-null float64
Reviewer_ID          359568 non-null float64
Time_Surveyed        379549 non-null object
Updated_At           379549 non-null object
Status               373229 non-null float64
Photo_URL            373217 non-null object
Structure            373227 non-null object
Use_                 373223 non-null object
Condition            261355 non-null object
Occupancy            261202 non-null object
Needs_B

In [271]:
MCM['CityParcelID2013'] = [x[:7] for x in MCM.CityParcelID2013]
MCM['D3_SurveyParcelID'] = [x[:7] for x in MCM.D3_SurveyParcelID]

In [272]:

MCM.drop(['GeoID10_BG','NAME10_Block','GEOID10_Block','Notes','Photo_URL','Time_Surveyed','Reviewer_ID','User_ID','Blext_ID','PCount','OBJECTID'], axis=1, inplace = True)

In [273]:
MCM.drop(['GEOID10_Tract','Updated_At'], axis=1, inplace = True)

In [274]:
blight = blight[blight.payment_status == 'PAID IN FULL']

In [275]:
columns_to_drop = [
                 
                 'ticket_number',
                 'agency_name',
                 'inspector_name',
                 'violator_name',
                
                 'violator_id',
                 'mailing_address_str_number',
                 'mailing_address_str_name',
                 'city',
                 'state',
                 'zip_code',
                 'non_us_str_code',
                 'country',
                 
                 'ticket_issued_time',
                 'hearing_date',
                 'hearing_time',
                 
                 'violation_code',
                 'violation_description',
                 'disposition',
                 'clean_up_cost',
                 'payment_amount',
                 
                 
                 'collection_status',
                 'violation_address',
                 
                 'oid',
                 'geom',
                 'fine_amount']

In [276]:
blight = blight.drop(columns=columns_to_drop)

In [277]:
blight.violation_date = pd.to_datetime(blight['violation_date']).dt.date
blight.judgment_date = pd.to_datetime(blight['judgment_date']).dt.date
blight.payment_date = pd.to_datetime(blight['payment_date']).dt.date

In [278]:
blight.rename({'Y':'lat', 'X':'lon', 'parcelno': 'parcel_id'}, axis=1, inplace=True)

In [279]:
blight['compliance'] = blight.apply(compliance, axis=1)

In [280]:
blight = blight.dropna(subset=['parcel_id'])

In [235]:
#blight['parcel_id'] = blight.apply(parse_parcel,axis=1)


In [236]:
blight['coordinates'] = blight.apply(coord_bligth,axis=1)

In [237]:
crimes = pd.read_csv('../data/Reported_Major_Crimes_2011_to_2014.csv')

In [238]:
crimes['lat'] = crimes.LOCATION.map(extract_lat_crime)
crimes['lon'] = crimes.LOCATION.map(extract_long_crime)

In [239]:
crimes =  crimes[ (crimes['lat'] < 45) | (crimes['lon'] < 85) ]

In [240]:
crimes['coordinates'] = crimes.apply(coord_bligth, axis=1)

In [241]:
crimes.INCIDENTDATE = pd.to_datetime(crimes['INCIDENTDATE']).dt.date

In [243]:
crimes.head()

,ROWNUM,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,NEIGHBORHOOD,CENSUSTRACT,LOCATION,ObjectId,lat,lon,coordinates
0,1311873,STOLEN VEHICLE,VEHICLE THEFT,24001,2011-01-02,0,1103.0,11.0,PERSHING,5066.0,"00 EUREKA/E LANTZ\n(42.4382, -83.0579)",1,42.4382,-83.0579,"(42.4382, -83.0579)"
1,1311874,AGGRAVATED ASSAULT,AGG/FEL ASSAULT - NON-FAMILY - OTHER WEAPON,13002,2011-01-02,11,709.0,7.0,LAFAYETTE PARK,5166.0,"E.LAFAYETTE01800\n(42.3394, -83.0295)",2,42.3394,-83.0295,"(42.3394, -83.0295)"
2,1311878,LARCENY,"LARCENY - FROM BUILDING (INCLUDES LIBRARY, OFF...",23003,2011-01-02,11,1105.0,11.0,OUTER DRIVE VAN DYKE,5051.0,"VANDYKE19900\n(42.4415, -83.0238)",3,42.4415,-83.0238,"(42.4415, -83.0238)"
3,1311879,LARCENY,LARCENY (OTHER),23007,2011-01-01,20,706.0,7.0,CHENE,5188.0,"JOS CAMPAU04100\n(42.3606, -83.0339)",4,42.3606,-83.0339,"(42.3606, -83.0339)"
4,1311883,STOLEN VEHICLE,VEHICLE THEFT,24001,2011-01-02,3,612.0,6.0,WARRENDALE,5457.0,"PIEDMONT06000\n(42.3328, -83.228)",5,42.3328,-83.2280,"(42.3328, -83.228)"


In [244]:
MCM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379549 entries, 0 to 379548
Data columns (total 20 columns):
D3_SurveyParcelID    379549 non-null object
CityParcelID2013     379549 non-null object
AddCombo             379549 non-null object
Address_Number       379549 non-null object
Address_Direction    379549 non-null object
Street               379549 non-null object
Zip                  379549 non-null object
Status               373229 non-null float64
Structure            373227 non-null object
Use_                 373223 non-null object
Condition            261355 non-null object
Occupancy            261202 non-null object
Needs_Boarding       261271 non-null object
Fire                 261284 non-null object
Dumping              373169 non-null object
Units                245276 non-null object
Improved             112058 non-null object
Maintained           112005 non-null object
PublicUse            7351 non-null object
NAMELSAD10_Tract     379549 non-null object
dtypes: fl

In [281]:
blight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59904 entries, 7 to 494929
Data columns (total 12 columns):
lon                        59904 non-null float64
lat                        59904 non-null float64
ticket_id                  59904 non-null int64
violation_street_number    59904 non-null int64
violation_street_name      59902 non-null object
violation_zip_code         22995 non-null object
violation_date             59904 non-null object
judgment_date              59904 non-null object
payment_date               59904 non-null object
payment_status             59904 non-null object
parcel_id                  59904 non-null object
compliance                 59904 non-null int64
dtypes: float64(2), int64(3), object(7)
memory usage: 5.9+ MB


In [286]:
blight['parcel_id'].value_counts()

1600000    157
2200987     96
1501156     80
1701632     79
0100026     63
          ... 
1300609      1
2101346      1
2206031      1
1602964      1
1301027      1
Name: parcel_id, Length: 16681, dtype: int64

In [288]:
MCM['D3_SurveyParcelID'].value_counts()

2200987    214
2210864    152
2211937    110
2212289    103
2206022    101
          ... 
2200476      1
2201811      1
1700418      1
1700987      1
2201485      1
Name: D3_SurveyParcelID, Length: 44032, dtype: int64

In [ ]:
blight['parcel_id'] = [x[:7] for x in blight.parcel_id]

In [250]:
def replace(row): 
    for row in re:
        if row['parcel_id'][0] == 'C' :
            re.drop(row)
    else:
        return str(row['parcel_id']);

In [258]:
blight['parcel_id'] = blight.replace('parcel_id')

In [259]:
blight.shape

(59904, 13)

In [260]:
MCM.shape

(379549, 20)

In [261]:
joined_df_1 = pd.merge(blight,MCM, how='left', left_on = 'parcel_id',right_on = 'CityParcelID2013',left_index=True)

In [262]:
joined_df_1.shape

(59904, 33)

In [263]:
joined_df_1.head()

,lon,lat,ticket_id,violation_street_number,violation_street_name,violation_zip_code,violation_date,judgment_date,payment_date,payment_status,...,Condition,Occupancy,Needs_Boarding,Fire,Dumping,Units,Improved,Maintained,PublicUse,NAMELSAD10_Tract
NaN,-83.019172,42.339196,18653,2900,JEFFERSON,NaN,2005-01-14,2005-02-23,2005-02-22,PAID IN FULL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,-82.942097,42.388525,18657,15439,MACK,NaN,2005-01-14,2005-02-18,2005-03-09,PAID IN FULL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,-82.946176,42.371778,18658,14214,JEFFERSON,NaN,2005-01-14,2005-02-04,2005-02-04,PAID IN FULL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,-83.045750,42.335121,18661,1310,BROADWAY,NaN,2005-01-14,2005-02-18,2007-06-12,PAID IN FULL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,-83.044467,42.335210,18664,343,MACOMB,NaN,2005-01-14,2005-03-23,2007-05-31,PAID IN FULL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
joined_df_1['Parcel_ID'].describe

<bound method NDFrame.describe of 0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
59899   NaN
59900   NaN
59901   NaN
59902   NaN
59903   NaN
Name: Parcel_ID, Length: 59904, dtype: float64>

In [190]:
def trim_lon(row):
    for row in re:
        x = str(row['lon'])
        x = x[:8]
        x = int(x);
def trim_lan(row):
    for row in re:
        y = str(row['lat'])
        y = y[:8]
        y = int(y);
    

In [192]:
joined_df_1['lon'] = joined_df_1.apply(trim_lon,'lon')
joined_df_1['lan'] = joined_df_1.trim_lan('lan')

ValueError: No axis named lon for object type <class 'pandas.core.frame.DataFrame'>

In [186]:
crimes.head()

,ROWNUM,CATEGORY,OFFENSEDESCRIPTION,STATEOFFENSEFILECLASS,INCIDENTDATE,HOUR,SCA,PRECINCT,NEIGHBORHOOD,CENSUSTRACT,LOCATION,ObjectId,lat,lon,coordinates
0,1311873,STOLEN VEHICLE,VEHICLE THEFT,24001,2011-01-02,0,1103.0,11.0,PERSHING,5066.0,"00 EUREKA/E LANTZ\n(42.4382, -83.0579)",1,42.4382,-83.0579,"(42.4382, -83.0579)"
1,1311874,AGGRAVATED ASSAULT,AGG/FEL ASSAULT - NON-FAMILY - OTHER WEAPON,13002,2011-01-02,11,709.0,7.0,LAFAYETTE PARK,5166.0,"E.LAFAYETTE01800\n(42.3394, -83.0295)",2,42.3394,-83.0295,"(42.3394, -83.0295)"
2,1311878,LARCENY,"LARCENY - FROM BUILDING (INCLUDES LIBRARY, OFF...",23003,2011-01-02,11,1105.0,11.0,OUTER DRIVE VAN DYKE,5051.0,"VANDYKE19900\n(42.4415, -83.0238)",3,42.4415,-83.0238,"(42.4415, -83.0238)"
3,1311879,LARCENY,LARCENY (OTHER),23007,2011-01-01,20,706.0,7.0,CHENE,5188.0,"JOS CAMPAU04100\n(42.3606, -83.0339)",4,42.3606,-83.0339,"(42.3606, -83.0339)"
4,1311883,STOLEN VEHICLE,VEHICLE THEFT,24001,2011-01-02,3,612.0,6.0,WARRENDALE,5457.0,"PIEDMONT06000\n(42.3328, -83.228)",5,42.3328,-83.2280,"(42.3328, -83.228)"


In [90]:
type(crimes)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234557 entries, 0 to 234977
Data columns (total 15 columns):
ROWNUM                   234557 non-null int64
CATEGORY                 234557 non-null object
OFFENSEDESCRIPTION       234557 non-null object
STATEOFFENSEFILECLASS    234557 non-null int64
INCIDENTDATE             234557 non-null object
HOUR                     234557 non-null int64
SCA                      233849 non-null float64
PRECINCT                 233849 non-null float64
NEIGHBORHOOD             233402 non-null object
CENSUSTRACT              217004 non-null float64
LOCATION                 234557 non-null object
ObjectId                 234557 non-null int64
lat                      234557 non-null float64
lon                      234557 non-null float64
coordinates              234557 non-null object
dtypes: float64(5), int64(4), object(6)
memory usage: 28.6+ MB


In [92]:
crimes.lon.value_counts()

-83.0670    192
-83.0671    188
-83.0413    178
-83.0845    177
-83.0846    176
           ... 
-82.8843      1
-83.4469      1
-82.9112      1
-83.3988      1
-83.5065      1
Name: lon, Length: 3820, dtype: int64

In [100]:
crimes['coordinates'].describe()

count                  234557
unique                 194544
top       (42.3677, -83.0846)
freq                       35
Name: coordinates, dtype: object

In [101]:
crimes['lon'].describe()

count    234557.000000
mean        -83.104547
std           0.297348
min         -84.912300
25%         -83.188500
50%         -83.112100
75%         -83.027700
max          14.200600
Name: lon, dtype: float64

In [116]:
joined_df_1.shape

(585797, 33)

In [104]:
joined_df_2 = joined_df_1[joined_df_1.payment_status == 'PAID IN FULL']

In [105]:
joined_df_2.shape

(585788, 33)